In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.color import gray2rgb
import cv2
from pandas import read_csv
import matplotlib.pyplot as plt
import glob

In [ ]:
%matplotlib inline

## Augmentation

In [ ]:
from utils import get_file_list
import glob
from train_val_split import read_facescrub_img_list
from plotting import plot_face_bb

### Train data split

In [ ]:
folder = '/media/shared/faceScrub/train_face_det/'
path = folder
actor_label_txt = '/media/shared/faceScrub/facescrub_actors.txt'
actress_label_txt = '/media/shared/faceScrub/facescrub_actresses.txt'
accept_pattern = '*/*.jpg'

In [ ]:
fnames, bboxes = read_facescrub_img_list(path, actor_label_txt, actress_label_txt, accept_pattern='*/*.jpg')

In [ ]:
len(fnames), len(bboxes)

### Data augment

In [ ]:
train_csv = 'train.csv'

In [ ]:
train_df = read_csv(train_csv, sep='\t')
X = np.asarray(train_df['name'].as_matrix())
y_str = train_df['bbox']
y_l = map(lambda k: [np.float32(v)
                     for v in k.split(',')], y_str)
y = np.asarray(y_l)

In [ ]:
x_1 = X[1]
y_1 = y[1, :]

In [ ]:
im = imread(x_1)
plot_face_bb(im, y_1, scale=False, path=False)
img = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)

In [ ]:
img_size = (256, 256, 3)
MAX_FACE_SIZE = 220
MIN_FACE_SIZE = 64

In [ ]:
h0, w0, _ = img_size
w1, h1, w2, h2 = y_1
wc, hc = (y_1[0] + y_1[2]) / 2, (y_1[1] + y_1[3]) / 2
face_width = (h2 - h1) / 2
print "Original center coords: (%.1f, %.1f)" % (wc, hc)
print "Face coords: (%.1f, %.1f) (%.1f, %.1f) and face width: %.0f" % (h1, w1, h2, w2, face_width * 2)

In [ ]:
rng = np.random.RandomState(seed=1234)

In [ ]:
# Possible scales of computation
high_scale = MAX_FACE_SIZE/2/face_width
low_scale = MIN_FACE_SIZE/2/face_width
print "Scales of computation: %.3f - %.3f" % (low_scale, high_scale)

scale_comp = rng.choice(np.arange(low_scale, high_scale, (high_scale-low_scale)/100), 1)[0]
new_face_width = round(face_width * scale_comp)
swc, shc = round(wc * scale_comp), round(hc * scale_comp)
sh1, sw1, sh2, sw2 = (shc-new_face_width, swc-new_face_width, 
                      shc+new_face_width, swc+new_face_width)
print "Chosen scale of computation: %.3f," % (scale_comp)
print "New face center: (%.1f, %.1f)\nface width: %.0f" % (
    shc, swc, new_face_width * 2)
res = cv2.resize(img, None, fx=scale_comp, fy=scale_comp)
h, w, _ = res.shape

In [ ]:
print "Face width: %.1f" % (new_face_width * 2)
plot_face_bb(res[:,:,::-1], y_1*scale_comp, scale=False, path=False)

In [ ]:
# Possible location of the face
min_pad = new_face_width + 30
lw, lh, hw, hh = (min(min_pad, w0 - min_pad), min(min_pad, h0 - min_pad), 
                  max(min_pad, w0 - min_pad), max(min_pad, h0 - min_pad))
print "Get face center in the given window: (%.1f, %.1f) (%.1f, %.1f)" % (lh, lw, hh, hw)
twc = rng.randint(lw, hw, 1)[0]
thc = rng.randint(lh, hh, 1)[0]
print "New center location: (%.1f, %.1f)" % (thc, twc)

In [ ]:
out = np.random.randint(0, high=255, size=img_size).astype(np.uint8)
sfh = shc - thc
tfh = int(0 if sfh > 0 else abs(sfh))
sfh = int(0 if sfh < 0 else sfh)
sfw = swc - twc
tfw = int(0 if sfw > 0 else abs(sfw))
sfw = int(0 if sfw < 0 else sfw)
print "source begin: (%.0f, %.0f) target begin: (%.0f, %.0f)" % (sfh, sfw, tfh, tfw)

In [ ]:
seh = shc - thc + h0
teh = int(h0 if seh <= h else h0 - seh + h)
seh = int(h if seh > h else seh)
sew = swc - twc + w0
tew = int(w0 if sew <= w else w0 - sew + w)
sew = int(w if sew > w else sew)
print "source end: (%.0f, %.0f) target end: (%.0f, %.0f)" % (seh, sew, teh, tew)

In [ ]:
ty_1 = np.array([twc-new_face_width, thc-new_face_width, twc+new_face_width, thc+new_face_width])
out[tfh:teh, tfw:tew, :] = res[sfh:seh, sfw:sew, :]
out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
plot_face_bb(out, ty_1, scale=False, path=False)
print "new face center: (%.0f, %.0f)" % (thc, twc)

In [ ]:
for i in glob.glob('*.jpg'):
    plt.figure()
    plt.imshow(imread(i))